In [1]:
from huggingface_hub import notebook_login,login
notebook_login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import PeftModel, PeftConfig
import torch
import os
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm

device='cuda'
model_name_or_path='bigscience/bloomz-7b1'
tokenizer_name_or_path='bigscience/bloomz-7b1'
dataset_name='twitter_complaints'
text_column='Tweet text'
label_column='text_label'
max_length=64
lr=1e-3
num_epochs=50
batch_size=8

In [ ]:
dataset = load_dataset("ought/raft", dataset_name)
# Get label names and replace underscore with spaces
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]

# Map label IDs to label names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)
print(dataset)
dataset["train"][0]

['Unlabeled', 'complaint', 'no complaint']
DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label', 'text_label'],
        num_rows: 3399
    })
})


{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [6]:
# Data Preprocessing 
tokenizer=AutoTokenizer.from_pretrained(model_name_or_path)

# Ensuring pad_token_id is defined
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id=tokenizer.eos_token_id
target_max_length=max([len(tokenizer(class_label)['input_ids'])for class_label in classes])
print(target_max_length)

def preprocess_function(examples):
    batch_size=len(examples[text_column])
    inputs=[f"{text_column}:{x} Label :" for x in examples[text_column]]
    targets=[str(x) for x in examples[label_column]]
    model_inputs=tokenizer(inputs)
    labels=tokenizer(targets, add_special_tokens=False)
    for i in range(batch_size):
        sample_input_ids=model_inputs['input_ids'][i]
        label_input_ids=labels['input_ids'][i] + [tokenizer.eos_token_id]
        # Append Targets to inputs
        model_inputs['input_ids'][i]= sample_input_ids + label_input_ids
        # Prepare the training labels
        labels['input_ids'][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs['attention_mask'][i]=[1] * len(model_inputs['input_ids'][i])


    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)

tokenizer_config.json:   0%|          | 0.00/223 [00:00<?, ?B/s]

c:\Users\ombha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ombha\.cache\huggingface\hub\models--bigscience--bloomz-7b1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

3


Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [7]:
def test_preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    model_inputs = tokenizer(inputs)
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
    return model_inputs


processed_datasets = dataset.map(
    test_preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

eval_dataset = processed_datasets["train"]
test_dataset = processed_datasets["test"]

eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
test_dataloader = DataLoader(test_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)
print(next(iter(eval_dataloader)))
print(next(iter(test_dataloader)))


Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

c:\Users\ombha\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'input_ids': tensor([[     3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3, 227985,   5484,    915,   2566, 169403,  15296,  36272,
            525,   3928,   1119,    632,   2670,   3968,  15270,  77658,    915,
            210],
        [     3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3,      3,      3,      3,      3,      3,      3,      3,      3,
              3, 227985,   5484,    915,   2566,  88653,   2321, 144017, 138861,
          59283,   1152,    613,   2632,  12120,      4,   5673,   1152,  321

In [24]:
from peft import PeftModel, PeftConfig

max_memory = {0: "1GIB", 1: "1GIB", 2: "2GIB", 3: "10GIB", "cpu": "30GB"}
peft_model_id = "OmBhandwalkar/twitter_complaints_bigscience_bloomz-7b1_LORA_CAUSAL_LM"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto", max_memory=max_memory)
model = PeftModel.from_pretrained(model, peft_model_id, device_map="auto", max_memory=max_memory)

OSError: The paging file is too small for this operation to complete. (os error 1455)

In [ ]:
model.hf_device_map

In [ ]:
model.eval()
i=89

inputs=tokenizer(f"{text_column} : {dataset['test'][i]['Tweet text']} Label : ",return_tensors='pt')
print(dataset['test'][i]['Tweet text'])
print(inputs)

with torch.no_grad():
    outputs=model.generate(input_ids=inputs['input_ids'], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(),skip_special_tokens=True))
